# Math AI Playground

- Evaluate numeric-only math expression step by step.
- Expression length cannot be more than 20 characters.
- Support `*` and `+` only, `-` and `()` support will be attempted later. Intended for the neural network to learn the rules of mathematics.
- Support integers only.

Character encoding:

| Character | Token    |
|-----------|----------|
| 0         | 0        |
| 1         | 1        |
| 2         | 2        |
| 3         | 3        |
| 4         | 4        |
| 5         | 5        |
| 6         | 6        |
| 7         | 7        |
| 8         | 8        |
| 9         | 9        |
| *         | 10       |
| +         | 11       |
| -         | 12       |
|  (space)  | 13       |

In [26]:
# Global imports

from typing import List, Tuple

import random
import re

import pandas as pd

import tensorflow as tf
import tensorflow_text as tf_text
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

RANDOM_SEED = 1

randomer = random.Random(RANDOM_SEED)

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
CHAR_TOKEN_MAP = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '*': 10,
    '+': 11,
    '-': 12,
    ' ': 13
}

TOKEN_CHAR_MAP = {}

for k, v in CHAR_TOKEN_MAP.items():
    TOKEN_CHAR_MAP[v] = k

TOKEN_CHAR_MAP

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: '*',
 11: '+',
 12: '-',
 13: ' '}

In [8]:
CHAR_VOCAB = list(CHAR_TOKEN_MAP)
CHAR_VOCAB

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '+', '-', ' ']

## Data generation

In [9]:
DIGIT_CHARS = list('0123456789')
OP_CHARS = list('*+')
EXPRESSION_LENGTH = 20
TOKEN_SPACE_SIZE = 16

def generate_initial_expression(rand: random.Random) -> str:
    initial_expression = [rand.choice(DIGIT_CHARS) if i % 3 < 2 else rand.choice(OP_CHARS) for i in range(14)]
    if initial_expression[0] == '0':
        initial_expression = initial_expression[1:]
    initial_expression = re.sub("([\*\+\-])0", lambda mo: mo.group(1), ''.join(initial_expression))
    
    return initial_expression

# Evaluate one step of the expression, return expression, False if no further
# steps can be made anymore.
def progress_expression_step(expression: str) -> Tuple[str, bool]:
    op = ''
    op_idx = -1

    if '*' in expression:
        # Multiplication takes precendence
        op = '*'
        op_idx = expression.find('*')
    else:
        m = re.search('\+', expression)
        if m is not None:
            op = expression[m.start()]
            op_idx = m.start()
    
    if op == ' ' or op_idx == -1:
        return expression, False
    
    start_idx = op_idx - 1
    end_idx = op_idx + 1

    while start_idx - 1 >= 0 and expression[start_idx - 1] not in OP_CHARS:
        start_idx -= 1
    
    while end_idx + 1 < len(expression) and expression[end_idx + 1] not in OP_CHARS:
        end_idx += 1

    num1 = int(expression[start_idx:op_idx])
    num2 = int(expression[op_idx+1:end_idx+1])

    calc_result = 0

    if op == '*':
        calc_result = num1 * num2
    elif op == '+':
        calc_result = num1 + num2
    elif op == '-':
        calc_result = num1 - num2

    before_result = expression[:start_idx]
    after_result = expression[end_idx+1:]

    if len(before_result) > 0 and calc_result < 0:
        if before_result[-1] == '-':
            before_result[-1] = '+'
        elif before_result[-1] == '+':
            before_result[-1] = '-'
    
    calc_result = abs(calc_result)

    return before_result + str(calc_result) + after_result, True
    

def generate_expression_with_steps(rand: random.Random) -> List[str]:
    initial_expression = generate_initial_expression(rand)
    ret = [initial_expression]
    while True:
        exp, has_further = progress_expression_step(ret[-1])
        if not has_further:
            break
        ret.append(exp)
    if int(eval(initial_expression)) != int(ret[-1]):
        raise ValueError("internal logic error, value evaluation is incorrect")
    return [l.ljust(20) for l in ret]

generate_expression_with_steps(randomer)

['29*41+77+31+6       ',
 '1189+77+31+6        ',
 '1266+31+6           ',
 '1297+6              ',
 '1303                ']

In [10]:
randomer = random.Random(1)

from_expression_train_file = open('data/from_expression_train.txt', "w")
to_expression_train_file = open('data/to_expression_train.txt', "w")

for i in range(10**6):
    steps = generate_expression_with_steps(randomer)
    for j in range(len(steps)-1):
        from_expression_train_file.write(steps[j] + "\n")
        to_expression_train_file.write(steps[j+1] + "\n")

from_expression_train_file.close()
to_expression_train_file.close()

from_expression_test_file = open('data/from_expression_test.txt', "w")
to_expression_test_file = open('data/to_expression_test.txt', "w")

for i in range(2 * 10**5):
    steps = generate_expression_with_steps(randomer)
    for j in range(len(steps)-1):
        from_expression_test_file.write(steps[j] + "\n")
        to_expression_test_file.write(steps[j+1] + "\n")

from_expression_test_file.close()
to_expression_test_file.close()

from_expression_validation_file = open('data/from_expression_validation.txt', "w")
to_expression_validation_file = open('data/to_expression_validation.txt', "w")

for i in range(2 * 10**5):
    steps = generate_expression_with_steps(randomer)
    for j in range(len(steps)-1):
        from_expression_validation_file.write(steps[j] + "\n")
        to_expression_validation_file.write(steps[j+1] + "\n")

from_expression_validation_file.close()
to_expression_validation_file.close()

In [11]:
%load_ext tensorboard

In [12]:
# Good reference https://www.tensorflow.org/tutorials/load_data/text

keys = [ord(c) for c in CHAR_VOCAB]
values = range(2, len(CHAR_VOCAB) + 2)

init = tf.lookup.KeyValueTensorInitializer(keys, values, key_dtype=tf.int64, value_dtype=tf.int64)

num_oov_buckets = 1
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

2022-10-20 21:33:57.061139: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-20 21:33:57.061277: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 21:33:57.061374: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.837GHz coreCount: 28 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 335.32GiB/s
2022-10-20 21:33:57.061402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-10-20 21:33:57.061415: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-10-20 21:33:57.061421: I tensorflow/stream_executor/platfor

In [13]:
from_expression_train_text_dataset = tf.data.TextLineDataset('data/from_expression_train.txt')
to_expression_train_text_dataset = tf.data.TextLineDataset('data/to_expression_train.txt')
from_expression_validation_text_dataset = tf.data.TextLineDataset('data/from_expression_validation.txt')
to_expression_validation_text_dataset = tf.data.TextLineDataset('data/to_expression_validation.txt')
from_expression_test_text_dataset = tf.data.TextLineDataset('data/from_expression_test.txt')
to_expression_test_text_dataset = tf.data.TextLineDataset('data/to_expression_test.txt')

In [14]:
tokenizer = tf_text.UnicodeCharTokenizer()

def tokenize(text):
    return tokenizer.tokenize(text)

tokenized_example_ds = from_expression_train_text_dataset.map(tokenize)

for text_batch in tokenized_example_ds.take(1):
    print("Tokens: ", text_batch[0])

2022-10-20 21:33:57.259114: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Tokens:  tf.Tensor(50, shape=(), dtype=int32)


2022-10-20 21:33:57.275590: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2496000000 Hz


In [15]:
def preprocess_text(text):
    standardized = tf_text.case_fold_utf8(text)
    tokenized = tokenizer.tokenize(standardized)
    tokenized = tf.cast(tokenized, tf.int64)
    vectorized = vocab_table.lookup(tokenized)
    return vectorized

example_text = next(iter(from_expression_train_text_dataset))
example_text.numpy()

preprocess_text(example_text)


<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([ 4, 11, 12,  6,  3, 13,  9,  9, 13,  5,  3, 13,  8, 15, 15, 15, 15,
       15, 15, 15])>

In [18]:
from_expression_train_encoded = from_expression_train_text_dataset.map(preprocess_text)
to_expression_train_encoded = to_expression_train_text_dataset.map(preprocess_text)
from_expression_test_encoded = from_expression_test_text_dataset.map(preprocess_text)
to_expression_test_encoded = to_expression_test_text_dataset.map(preprocess_text)
from_expression_validation_encoded = from_expression_validation_text_dataset.map(preprocess_text)
to_expression_validation_encoded = to_expression_validation_text_dataset.map(preprocess_text)

In [23]:
train_data = tf.data.Dataset.zip((from_expression_train_encoded, to_expression_train_encoded))
train_data

<ZipDataset shapes: ((None,), (None,)), types: (tf.int64, tf.int64)>

In [24]:
validation_data = tf.data.Dataset.zip((from_expression_validation_encoded, to_expression_validation_encoded))
validation_data

<ZipDataset shapes: ((None,), (None,)), types: (tf.int64, tf.int64)>

In [27]:
test_data = tf.data.Dataset.zip((from_expression_test_encoded, to_expression_test_encoded))
test_data

<ZipDataset shapes: ((None,), (None,)), types: (tf.int64, tf.int64)>

In [28]:
BATCH_SIZE = 32
train_data_batched = train_data.padded_batch(BATCH_SIZE)
validation_data_batched = validation_data.padded_batch(BATCH_SIZE)

In [22]:
next(iter(from_expression_train_encoded))

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([ 4, 11, 12,  6,  3, 13,  9,  9, 13,  5,  3, 13,  8, 15, 15, 15, 15,
       15, 15, 15])>

In [42]:
PADDED_VOCAB_SIZE = len(CHAR_VOCAB) + 2

print(PADDED_VOCAB_SIZE)

embedding_layer = layers.Embedding(PADDED_VOCAB_SIZE, 64, mask_zero=True, input_length=EXPRESSION_LENGTH)

embedding_layer(next(iter(train_data.as_numpy_iterator()))[0])

16


<tf.Tensor: shape=(20, 64), dtype=float32, numpy=
array([[ 0.0298552 , -0.03905521,  0.04754081, ..., -0.00102699,
         0.00503007, -0.00370364],
       [-0.02448289,  0.01715047,  0.01277838, ..., -0.0447557 ,
        -0.04201703, -0.03396982],
       [-0.01443779, -0.04983015,  0.00855494, ...,  0.04166007,
        -0.00626879,  0.01580865],
       ...,
       [ 0.04295739,  0.01605911,  0.01924599, ...,  0.0175524 ,
        -0.03528353, -0.04884288],
       [ 0.04295739,  0.01605911,  0.01924599, ...,  0.0175524 ,
        -0.03528353, -0.04884288],
       [ 0.04295739,  0.01605911,  0.01924599, ...,  0.0175524 ,
        -0.03528353, -0.04884288]], dtype=float32)>